# **HW1: Regression**
In *assignment 1*, you need to finish:

1.  Basic Part: Implement two regression models to predict the Systolic blood pressure (SBP) of a patient. You will need to implement **both Matrix Inversion and Gradient Descent**.


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implement one regression model to predict the SBP of multiple patients in a different way than the basic part. You can choose **either** of the two methods for this part.

# **1. Basic Part (55%)**
In the first part, you need to implement the regression to predict SBP from the given DBP


## 1.1 Matrix Inversion Method (25%)


*   Save the prediction result in a csv file **hw1_basic_mi.csv**
*   Print your coefficient


### *Import Packages*

> Note: You **cannot** import any other package

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

### *Global attributes*
Define the global attributes

In [28]:
training_dataroot = 'hw1_basic_training.csv' # Training data file file named as 'hw1_basic_training.csv'
testing_dataroot = 'hw1_basic_testing.csv'   # Testing data file named as 'hw1_basic_training.csv'
output_dataroot = 'hw1_basic_mi.csv' # Output file will be named as 'hw1_basic.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be 20 * 1 matrix and saved as numpy array
                      # The format of each row should be ['sbp']

You can add your own global attributes here


### *Load the Input File*
First, load the basic input file **hw1_basic_training.csv** and **hw1_basic_testing.csv**

Input data would be stored in *training_datalist* and *testing_datalist*

In [29]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = np.array(list(csv.reader(csvfile)))

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = np.array(list(csv.reader(csvfile)))

### *Implement the Regression Model*

> Note: It is recommended to use the functions we defined, you can also define your own functions


#### Step 1: Split Data
Split data in *training_datalist* into training dataset and validation dataset
* Validation dataset is used to validate your own model without the testing data



In [30]:
def SplitData(debug):
  #分成training和validation
  data = training_datalist[1:]


  #分割dataset
  split_ratio = 0.8
  number_samples = len(data)


  train_number_samples = int(split_ratio * number_samples)
  if(debug==True):
    print(f"train_number_samples = {train_number_samples}")

  training_data = data[:train_number_samples]
  validation_data = data[train_number_samples:]

  #轉換成numpy格式
  training_data = np.array(training_data)
  validation_data = np.array(validation_data)

  return training_data, validation_data



#### Step 2: Preprocess Data
Handle the unreasonable data
> Hint: Outlier and missing data can be handled by removing the data or adding the values with the help of statistics  

In [31]:
def PreprocessData(debug, data):

  #basic part沒有missing data
  #但有幾筆資料明顯不合理，可以從圖片中看出
  #這邊的預處理要替換過於不合理的資料
  dbp = [int(row[0]) for row in data]
  sbp = [int(row[1]) for row in data]

  if(debug == True):
    print(f"dbp before =  {dbp}")

  threshold = 2.0

  # 使用平均值來填
  dbp_mean = np.nanmean(dbp)
  dbp_std = np.nanstd(dbp)
  dbp = [val if not np.isnan(val) and abs(val - dbp_mean) <= threshold * dbp_std else dbp_mean for val in dbp]
  dbp = np.array(dbp)

  sbp_mean = np.nanmean(sbp)
  sbp_std = np.nanstd(sbp)
  sbp = [val if not np.isnan(val) and abs(val - sbp_mean) <= threshold * sbp_std else sbp_mean for val in sbp]
  sbp = np.array(sbp)



  if(debug == True):
    print(f"dbp_mean =  {dbp_mean}")
    print(f"dbp_std =  {dbp_std}")
    print(f"sbp_mean =  {sbp_mean}")
    print(f"sbp_std =  {sbp_std}")
    number_samples = len(data)
    print("number of samples = " + str(number_samples))


  #視覺化
  if(debug == True):
    #X = [int(row[0]) for row in new_data]
    #Y = [int(row[1]) for row in new_data]

    plt.figure(figsize=(8, 6))
    plt.scatter(dbp, sbp, alpha=0.5)
    plt.title('dbp vs sbp')
    plt.xlabel('dbp')
    plt.ylabel('sbp')
    plt.grid(True)


  return dbp, sbp




#### Step 3: Implement Regression
> use Matrix Inversion to finish this part




In [32]:
def MatrixInversion(debug, Xl_dbp, Y_sbp):
  w0 = 70.0
  w1 = 0.0 #dbp

  new_intercept_value = 1.0

  X = np.column_stack((np.full(Xl_dbp.shape, new_intercept_value), Xl_dbp))

  w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y_sbp)
  w0, w1 = w[0], w[1]


  return w0, w1



#### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*
The final *output_datalist* should look something like this
> [ [100], [80], ... , [90] ] where each row contains the predicted SBP

In [33]:
def MakePrediction(debug, w0, w1):

  test_data = testing_datalist[1:]
  X_test = [int(row[0]) for row in test_data]

  X_test = np.column_stack((np.ones(len(X_test)), X_test))

  # 預測
  sbp_predicted = w0 + w1 * X_test[:, 1]
  sbp_predicted = np.round(sbp_predicted).astype(int)

  if(debug==True):
    print("sbp_predicted = ", sbp_predicted)

  return sbp_predicted


def MAPE(debug, w, X1_dbp, Y_sbp):

  X1_dbp = np.column_stack((np.ones(len(X1_dbp)), X1_dbp))

  Y_pred = w[0] + w[1] * X1_dbp[:, 1]
  Y_pred = np.round(Y_pred).astype(int)

  n = len(Y_sbp)
  mape = (1 / n) * np.sum(np.abs((Y_sbp - Y_pred) / Y_sbp)) * 100

  # print
  if(debug == True):
    print("MAPE：", mape)




#### Step 5: Train Model and Generate Result

> Notice: **Remember to output the coefficients of the model here**, otherwise 5 points would be deducted
* If your regression model is *3x^2 + 2x^1 + 1*, your output would be:
```
3 2 1
```





In [34]:
# 分割資料
training_data, validation_data = SplitData(False)

#print(f"training data len = {len(training_data)}")
#print(f"Validation data len = {len(validation_data)}")

# 預處理資料
dbp, sbp = PreprocessData(False, training_data)
dbp_val, sbp_val = PreprocessData(False, validation_data)


# 跑模型計算
w0, w1 = MatrixInversion(False, dbp,  sbp)

# 印出weights
print(w1, w0)

# 測試MAPE
mape_value = MAPE(True, [w0, w1], dbp_val, sbp_val)

# 預測
output_datalist = MakePrediction(True, w0, w1)

0.9140180543173676 54.038797158228704
MAPE： 5.617740912398577
sbp_predicted =  [140 131 130 146 111 145 137 137 132 126 125 128 119 125 139 134 151 140
 142 148]


### *Write the Output File*
Write the prediction to output csv
> Format: 'sbp'




In [35]:
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in output_datalist:
    writer.writerow([row])

## 1.2 Gradient Descent Method (30%)


*   Save the prediction result in a csv file **hw1_basic_gd.csv**
*   Output your coefficient update in a csv file **hw1_basic_coefficient.csv**
*   Print your coefficient





### *Global attributes*

In [36]:
output_dataroot = 'hw1_basic_gd.csv' # Output file will be named as 'hw1_basic.csv'
coefficient_output_dataroot = 'hw1_basic_coefficient.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be 20 * 1 matrix and saved as numpy array
                      # The format of each row should be ['sbp']

coefficient_output = [] # Your coefficient update during gradient descent
                   # Should be a (number of iterations * number_of coefficient) matrix
                   # The format of each row should be ['w0', 'w1', ...., 'wn']

Your own global attributes

In [37]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = np.array(list(csv.reader(csvfile)))

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = np.array(list(csv.reader(csvfile)))

### *Implement the Regression Model*


#### Step 1: Split Data

In [38]:
def SplitData(debug):
  #分成training和validation
  data = training_datalist[1:]


  #分割dataset
  split_ratio = 0.8
  number_samples = len(data)


  train_number_samples = int(split_ratio * number_samples)
  if(debug==True):
    print(f"train_number_samples = {train_number_samples}")

  training_data = data[:train_number_samples]
  validation_data = data[train_number_samples:]

  #轉換成numpy格式
  training_data = np.array(training_data)
  validation_data = np.array(validation_data)

  return training_data, validation_data


#### Step 2: Preprocess Data

In [39]:
def PreprocessData(debug, data):

  #basic part沒有missing data
  #但有幾筆資料明顯不合理，可以從圖片中看出
  #這邊的預處理要替換過於不合理的資料
  dbp = [int(row[0]) for row in data]
  sbp = [int(row[1]) for row in data]

  if(debug == True):
    print(f"dbp before =  {dbp}")

  threshold = 2.0

  # 使用平均值來填
  dbp_mean = np.nanmean(dbp)
  dbp_std = np.nanstd(dbp)
  dbp = [val if not np.isnan(val) and abs(val - dbp_mean) <= threshold * dbp_std else dbp_mean for val in dbp]
  dbp = np.array(dbp)

  sbp_mean = np.nanmean(sbp)
  sbp_std = np.nanstd(sbp)
  sbp = [val if not np.isnan(val) and abs(val - sbp_mean) <= threshold * sbp_std else sbp_mean for val in sbp]
  sbp = np.array(sbp)



  if(debug == True):
    print(f"dbp_mean =  {dbp_mean}")
    print(f"dbp_std =  {dbp_std}")
    print(f"sbp_mean =  {sbp_mean}")
    print(f"sbp_std =  {sbp_std}")
    number_samples = len(data)
    print("number of samples = " + str(number_samples))


  #視覺化
  if(debug == True):
    #X = [int(row[0]) for row in new_data]
    #Y = [int(row[1]) for row in new_data]

    plt.figure(figsize=(8, 6))
    plt.scatter(dbp, sbp, alpha=0.5)
    plt.title('dbp vs sbp')
    plt.xlabel('dbp')
    plt.ylabel('sbp')
    plt.grid(True)


  return dbp, sbp





#### Step 3: Implement Regression
> use Gradient Descent to finish this part

In [40]:
def GradientDescent(debug, learning_rate, num_iterations, X1_dbp, Y_sbp):
  #初始weights
  w0 = 50.0
  w1 = 0.0 #dbp


  coefficient = []

  #iteration訓練
  for iteration in range(num_iterations):
    coefficient.append([w0, w1])

    #計算預測值
    Y_pred = w0 + X1_dbp * w1

    # training loss
    train_loss = np.mean((Y_sbp - Y_pred) ** 2)


    # 計算loss function gradient
    gradient_w0 = -np.mean(Y_sbp - Y_pred)
    gradient_w1 = -np.mean((Y_sbp - Y_pred) * X1_dbp)


    # 更新参数
    w0 = w0 - learning_rate * gradient_w0
    w1 = w1 - learning_rate * gradient_w1



    # print結果
    if(debug==True):
      print(f"Iteration {iteration}: Train Loss = {train_loss}, coefficients = {w0, w1}")



  # 打印最终的模型参数
  if(debug==True):
    print("最终的模型参数：")
    print(f"w0 = {w0}")
    print(f"w1 = {w1}")



  return w0, w1, coefficient


#### Step 4: Make Prediction

Make prediction of testing dataset and store the values in *output_datalist*
The final *output_datalist* should look something like this
> [ [100], [80], ... , [90] ] where each row contains the predicted SBP

Remember to also store your coefficient update in *coefficient_output*
The final *coefficient_output* should look something like this
> [ [1, 0, 3, 5], ... , [0.1, 0.3, 0.2, 0.5] ] where each row contains the [w0, w1, ..., wn] of your coefficient





In [41]:
def MakePrediction(debug, w0, w1):

  test_data = testing_datalist[1:]
  X_test = [int(row[0]) for row in test_data]

  X_test = np.column_stack((np.ones(len(X_test)), X_test))

  # 預測
  sbp_predicted = w0 + w1 * X_test[:, 1]
  sbp_predicted = np.round(sbp_predicted).astype(int)

  if(debug==True):
    print("sbp_predicted = ", sbp_predicted)

  return sbp_predicted


def MAPE(debug, w, X1_dbp, Y_sbp):

  X1_dbp = np.column_stack((np.ones(len(X1_dbp)), X1_dbp))

  Y_pred = w[0] + w[1] * X1_dbp[:, 1]
  Y_pred = np.round(Y_pred).astype(int)

  n = len(Y_sbp)
  mape = (1 / n) * np.sum(np.abs((Y_sbp - Y_pred) / Y_sbp)) * 100

  # print
  if(debug == True):
    print("MAPE：", mape)




#### Step 5: Train Model and Generate Result

> Notice: **Remember to output the coefficients of the model here**, otherwise 5 points would be deducted
* If your regression model is *3x^2 + 2x^1 + 1*, your output would be:
```
3 2 1
```



In [42]:
# 每個function第一個參數是debug，設True代表要看debug資訊
# 分割資料
training_data, validation_data = SplitData(False)


# 預處理資料
dbp, sbp = PreprocessData(False, training_data)
dbp_val, sbp_val = PreprocessData(False, validation_data)

#學習速率與iterations
learning_rate = 0.0001
num_iterations = 100

# 跑模型計算
w0, w1, coefficient_output = GradientDescent(False, learning_rate, num_iterations, dbp, sbp)

# 印出weights
print(w1, w0)

print(f"coefficient_output = {coefficient_output}")

# 測試MAPE
mape_value = MAPE(True, [w0, w1], dbp_val, sbp_val)

# 預測
output_datalist = MakePrediction(True, w0, w1)

0.9618903711182225 50.01241816065527
coefficient_output = [[50.0, 0.0], [50.00790401108058, 0.6639540043241295], [50.0103590436974, 0.8696128223159314], [50.011126262229595, 0.9333152960169667], [50.01137068250423, 0.9530469747253534], [50.011453166674734, 0.9591587562609438], [50.011485491328834, 0.9610517921717942], [50.011502279120755, 0.9616380770537184], [50.01151425437309, 0.9618195974859888], [50.01152473893097, 0.9618757426509742], [50.01153476173165, 0.9618930529884313], [50.01154464148712, 0.9618983342882255], [50.011554476918, 0.9618998896092772], [50.011564298602885, 0.961900290813404], [50.01157411601345, 0.9619003345314947], [50.01158393208354, 0.961900267518942], [50.0115937477219, 0.9619001662079112], [50.01160356321003, 0.9619000542731674], [50.01161337863511, 0.9618999390479384], [50.011623194024146, 0.9618998228036821], [50.011633009385505, 0.9618997062439799], [50.011642824721775, 0.961899589586765], [50.011652640033766, 0.961899472899542], [50.01166245532172, 0.961

### *Write the Output File*

Write the prediction to output csv
> Format: 'sbp'

**Write the coefficient update to csv**
> Format: 'w0', 'w1', ..., 'wn'
>*   The number of columns is based on your number of coefficient
>*   The number of row is based on your number of iterations

In [43]:
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in output_datalist:
    writer.writerow([row])

with open(coefficient_output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in coefficient_output:
    writer.writerow(row)

# **2. Advanced Part (40%)**
In the second part, you need to implement the regression in a different way than the basic part to help your predictions of multiple patients SBP.

You can choose **either** Matrix Inversion or Gradient Descent method.

The training data will be in **hw1_advanced_training.csv** and the testing data will be in **hw1_advanced_testing.csv**.

Output your prediction in **hw1_advanced.csv**

Notice:
> You cannot import any other package other than those given



### Input the training and testing dataset

In [44]:
training_dataroot = 'hw1_advanced_training.csv' # Training data file file named as 'hw1_basic_training.csv'
testing_dataroot = 'hw1_advanced_testing.csv'   # Testing data file named as 'hw1_basic_training.csv'
output_dataroot = 'hw1_advanced.csv' # Output file will be named as 'hw1_basic.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be 220 * 1 matrix and saved as numpy array
                      # The format of each row should be ['sbp']



### Your Implementation

In [45]:
#key: 用多項參數來預測sbp，沒有dbp可以用
#就先試著用sbp = w0 + w1 * temperature + w2 * heartrate + w3 * resprate + w4 * o2sat吧

#1. 先做資料讀取
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = np.array(list(csv.reader(csvfile)))

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = np.array(list(csv.reader(csvfile)))

#print(testing_datalist)

#2. 預處理資料，先用1位病人的資料當範例就好，想想看怎麼處理空的資料
#遇到空值的資料填成平均值，outliers也填成平均值
#應該先不用分validation set

#subject_id = []
#charttime = []
#temperature = []
#heartrate = []
#resprate = []
#o2sat = []
#sbp = []

#分成training和validation
def ad_SplitData(debug, id, ratio):
  data = training_datalist[1:]

  #指定病人
  selected_subject_id = str(id)
  selected_data = [row for row in data if row[0] == selected_subject_id]


  #分割dataset
  split_ratio = ratio
  number_samples = len(selected_data)


  train_number_samples = int(split_ratio * number_samples)
  if(debug==True):
    print(f"train_number_samples = {train_number_samples}")

  training_data = selected_data[:train_number_samples]
  validation_data = selected_data[train_number_samples:]

  #轉換成numpy格式
  training_data = np.array(training_data)
  validation_data = np.array(validation_data)

  return training_data, validation_data


def ad_PreprocessData(debug, data):
  #header = training_datalist[0]
  #data = training_datalist[1:]

  subject_id = [int(row[0]) for row in data]
  charttime = [str(row[1]) for row in data]
  temperature = [float(row[2]) if row[2] else np.nan for row in data]
  heartrate = [float(row[3]) if row[3] else np.nan for row in data]
  resprate = [float(row[4]) if row[4] else np.nan for row in data]
  o2sat = [float(row[5]) if row[5] else np.nan for row in data]
  sbp = [float(row[6]) if row[6] else np.nan for row in data]

  if(debug == True):
    print(f"temperature before =  {temperature}")

  threshold = 2.0

  # 使用平均值來填
  temperature_mean = np.nanmean(temperature)
  temperature_std = np.nanstd(temperature)
  #temperature = [val if not np.isnan(val) else temperature_mean for val in temperature]
  temperature = [val if not np.isnan(val) and abs(val - temperature_mean) <= threshold * temperature_std else temperature_mean for val in temperature]
  temperature = np.array(temperature)

  heartrate_mean = np.nanmean(heartrate)
  heartrate_std = np.nanstd(heartrate)
  heartrate = [val if not np.isnan(val) and abs(val - heartrate_mean) <= threshold * heartrate_std else heartrate_mean for val in heartrate]
  heartrate = np.array(heartrate)

  resprate_mean = np.nanmean(resprate)
  resprate_std = np.nanstd(resprate)
  resprate = [val if not np.isnan(val) and abs(val - resprate_mean) <= threshold * resprate_std else resprate_mean for val in resprate]
  resprate = np.array(resprate)

  o2sat_mean = np.nanmean(o2sat)
  o2sat_std = np.nanstd(o2sat)
  o2sat = [val if not np.isnan(val) and abs(val - o2sat_mean) <= threshold * o2sat_std else o2sat_mean for val in o2sat]
  o2sat = np.array(o2sat)

  sbp_mean = np.nanmean(sbp)
  sbp_std = np.nanstd(sbp)
  sbp = [val if not np.isnan(val) and abs(val - sbp_mean) <= threshold * sbp_std else sbp_mean for val in sbp]
  sbp = np.array(sbp)



  if(debug == True):
    print(f"temperature_mean =  {temperature_mean}")
    print(f"heartrate_mean =  {heartrate_mean}")
    print(f"resprate_mean =  {resprate_mean}")
    print(f"o2sat_mean =  {o2sat_mean}")
    print(f"sbp_mean =  {sbp_mean}")
    print(f"temperature after =  {temperature}")
    number_samples = len(data)
    print("number of samples = " + str(number_samples))

  return temperature, heartrate, resprate, o2sat, sbp


#3. Gradient descent
def GradientDescent(debug, learning_rate, num_iterations, X1_temperature, X2_heartrate, X3_resprate, X4_o2sat, Y_sbp):
  #初始weights
  w0 = 70.0
  w1 = 0.0 #temperature
  w2 = 0.0 #heartrate
  w3 = 0.0 # resprate
  w4 = 0.0 #o2sat


  #coefficients = []

  #iteration訓練
  for iteration in range(num_iterations):
    #coefficients.append([w0, w1])

    #計算預測值
    #Y_pred = w0 + X1_temperature*w1 + X2_heartrate*w2 + X3_resprate*w3 + X4_o2sat*w4
    Y_pred = w0 + X2_heartrate*w2 + X3_resprate*w3

    # training loss
    train_loss = np.mean((Y_sbp - Y_pred) ** 2)


    # 計算loss function gradient
    gradient_w0 = -np.mean(Y_sbp - Y_pred)
    ##gradient_w1 = -np.mean((Y_sbp - Y_pred) * X1_temperature)
    gradient_w2 = -np.mean((Y_sbp - Y_pred) * X2_heartrate)
    gradient_w3 = -np.mean((Y_sbp - Y_pred) * X3_resprate)
    ##gradient_w4 = -np.mean((Y_sbp - Y_pred) * X4_o2sat)

    # 更新参数
    w0 = w0 - learning_rate * gradient_w0
    ##w1 = w1 - learning_rate * gradient_w1
    w2 = w2 - learning_rate * gradient_w2
    w3 = w3 - learning_rate * gradient_w3
    ##w4 = w4 - learning_rate * gradient_w4

    # print結果
    if(debug==True):
      print(f"Iteration {iteration}: Train Loss = {train_loss}, coefficients = {w0, w1, w2, w3, w4}")


  return w0, w1, w2, w3, w4


# matrix inversion
def MatrixInversion(debug, X1_temperature, X2_heartrate, X3_resprate, X4_o2sat, Y_sbp, id):
  w0 = 70.0
  w1 = 0.0 #temperature
  w2 = 0.0 #heartrate
  w3 = 0.0 # resprate
  w4 = 0.0 #o2sat

  new_intercept_value = 1.0

  #X = np.column_stack((np.ones(len(X1_temperature)), X1_temperature, X2_heartrate, X3_resprate, X4_o2sat))

  #X = np.column_stack((np.full(X1_temperature.shape, new_intercept_value), X1_temperature, X2_heartrate, X3_resprate, X4_o2sat))
  if(id == 14699420 or id == 15437705):
    X = np.column_stack((np.full(X1_temperature.shape, new_intercept_value), X2_heartrate*2 , X3_resprate))
  else:
    X = np.column_stack((np.full(X1_temperature.shape, new_intercept_value), X2_heartrate , X3_resprate))


  w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y_sbp)
  w0, w2, w3 = w[0], w[1], w[2]


  return w0, w1, w2, w3, w4

#視覺化
def DrawData(temperature, heartrate, resprate, o2sat, sbp):
  #temperature vs sbp
  plt.figure(figsize=(8, 6))
  plt.scatter(temperature, sbp, alpha=0.5)
  plt.title('temperature vs sbp')
  plt.xlabel('temperature')
  plt.ylabel('sbp')
  plt.grid(True)

  #heartrate vs sbp
  plt.figure(figsize=(8, 6))
  plt.scatter(heartrate, sbp, alpha=0.5)
  plt.title('heartrate vs sbp')
  plt.xlabel('heartrate')
  plt.ylabel('sbp')
  plt.grid(True)

  #resprate vs sbp
  plt.figure(figsize=(8, 6))
  plt.scatter(resprate, sbp, alpha=0.5)
  plt.title('resprate vs sbp')
  plt.xlabel('resprate')
  plt.ylabel('sbp')
  plt.grid(True)

  #o2sat vs sbp
  plt.figure(figsize=(8, 6))
  plt.scatter(o2sat, sbp, alpha=0.5)
  plt.title('o2sat vs sbp')
  plt.xlabel('o2sat')
  plt.ylabel('sbp')
  plt.grid(True)



#計算MAPE
def MAPE(debug, w, X1_temperature, X2_heartrate, X3_resprate, X4_o2sat, Y_sbp):

  X1_temperature = np.column_stack((np.ones(len(X1_temperature)), X1_temperature))
  X2_heartrate = np.column_stack((np.ones(len(X2_heartrate)), X2_heartrate))
  X3_resprate = np.column_stack((np.ones(len(X3_resprate)), X3_resprate))
  X4_o2sat = np.column_stack((np.ones(len(X4_o2sat)), X4_o2sat))

  #預測
  Y_pred = w[0] + w[1] * X1_temperature[:, 1] + w[2] * X2_heartrate[:, 1] + w[3] * X3_resprate[:, 1] + w[4] * X4_o2sat[:, 1]
  #Y_pred = np.round(Y_pred).astype(int)

  n = len(Y_sbp)
  mape = (1 / n) * np.sum(np.abs((Y_sbp - Y_pred) / Y_sbp)) * 100

  # 打印 MAPE
  if(debug == True):
    print("MAPE：", mape)

  return mape

# 預測
def ad_MakePrediction(debug, w, patients):

  data = testing_datalist[1:]
  sbp_pred = []

  for id in patients:
    #指定病人
    selected_subject_id = str(id)
    selected_data = [row for row in data if row[0] == selected_subject_id]

    temperature = [float(row[2]) if row[2] else 0.0 for row in selected_data]
    heartrate = [float(row[3]) if row[3] else 0.0 for row in selected_data]
    resprate = [float(row[4]) if row[4] else 0.0 for row in selected_data]
    o2sat = [float(row[5]) if row[5] else 0.0 for row in selected_data]
    sbp = [float(row[6]) if row[6] else 0.0 for row in selected_data]

    temperature = np.array(temperature)
    heartrate = np.array(heartrate)
    resprate = np.array(resprate)
    o2sat = np.array(o2sat)
    sbp = np.array(sbp)


    X1_temperature = np.column_stack((np.ones(len(temperature)), temperature))
    X2_heartrate = np.column_stack((np.ones(len(heartrate)), heartrate))
    X3_resprate = np.column_stack((np.ones(len(resprate)), resprate))
    X4_o2sat = np.column_stack((np.ones(len(o2sat)), o2sat))

    #預測
    Y_pred = w[id][0] + w[id][1] * X1_temperature[:, 1] + w[id][2] * X2_heartrate[:, 1] + w[id][3] * X3_resprate[:, 1] + w[id][4] * X4_o2sat[:, 1]
    Y_pred = np.round(Y_pred).astype(int)

    sbp_pred.append(Y_pred)

    if(debug==True):
      print("Y_pred = ", Y_pred)

  sbp_predicted = [item for sublist in sbp_pred for item in sublist]
  #sbp_predicted = np.array(sbp_predicted)

  return sbp_predicted


# 啟用 --------------------------------------------------------------------------

#學習速率與iterations
learning_rate = 0.0001
num_iterations = 10000


patients = [11526383, 12923910, 14699420, 15437705, 15642911, 16298357, 17331999
, 17593883, 18733920, 18791093, 19473413]

MAPEs = []
Weights = dict()

# 測試用
#training_data, validation_data = ad_SplitData(False, 15437705, 0.8)
#temperature, heartrate, resprate, o2sat, sbp = ad_PreprocessData(False, training_data)
#DrawData(temperature, heartrate, resprate, o2sat, sbp)

Debug = False

for id in patients:

  # 分割資料
  training_data, validation_data = ad_SplitData(False, id, 0.8)

  if(Debug == True):
    print(f"----------------patient { id }-----------------")
    print(f"training data len = {len(training_data)}")
    print(f"Validation data len = {len(validation_data)}")


  # 預處理資料
  temperature, heartrate, resprate, o2sat, sbp = ad_PreprocessData(False, training_data)
  temperature_val, heartrate_val, resprate_val, o2sat_val, sbp_val = ad_PreprocessData(False, validation_data)

  # 畫出圖片來看
  #DrawData(temperature, heartrate, resprate, o2sat, sbp)

  # 跑模型計算
  #w0, w1, w2, w3, w4 = GradientDescent(False, learning_rate, num_iterations, temperature, heartrate, resprate, o2sat, sbp)
  w0, w1, w2, w3, w4 = MatrixInversion(True, temperature, heartrate, resprate, o2sat, sbp, id)

  # 測試MAPE
  mape_value = MAPE(False, [w0, w1, w2, w3, w4], temperature_val, heartrate_val, resprate_val, o2sat_val, sbp_val)

  MAPEs.append(mape_value)
  # print

  if(Debug == True):
    print("最终的模型参数：")
    print(f"w0 = {w0}")
    print(f"w1 = {w1}")
    print(f"w2 = {w2}")
    print(f"w3 = {w3}")
    print(f"w4 = {w4}")

    print(f"MAPE: {mape_value}")

  Weights[id] = [w0, w1, w2, w3, w4]

print("-----------------------------------------------")
print(f"average MAPE = {sum(MAPEs) / len(MAPEs)}")


# 預測
#temperature_test, heartrate_test, resprate_test, o2sat_test, sbp_test = ad_PreprocessData(False, testing_datalist[1:])
output_datalist = ad_MakePrediction(False, Weights, patients)
print(output_datalist)
#print(Weights)


-----------------------------------------------
average MAPE = 10.414305482335003
[152, 154, 161, 150, 162, 151, 147, 157, 146, 162, 157, 154, 160, 160, 153, 148, 159, 160, 146, 151, 147, 141, 140, 138, 143, 141, 146, 142, 145, 139, 142, 141, 139, 140, 152, 144, 141, 139, 149, 139, 114, 110, 115, 116, 117, 108, 114, 123, 114, 113, 119, 113, 123, 123, 116, 126, 129, 126, 127, 114, 151, 152, 152, 154, 143, 143, 143, 150, 150, 152, 153, 149, 144, 149, 151, 150, 148, 151, 146, 146, 126, 126, 127, 127, 125, 126, 125, 124, 126, 126, 124, 126, 126, 127, 125, 127, 126, 125, 126, 126, 131, 132, 131, 134, 131, 132, 136, 134, 130, 132, 136, 133, 132, 129, 134, 132, 130, 129, 134, 132, 119, 116, 117, 119, 119, 117, 115, 116, 117, 117, 117, 119, 118, 120, 119, 117, 117, 119, 118, 116, 124, 123, 127, 124, 124, 122, 124, 131, 130, 127, 119, 124, 124, 123, 124, 123, 123, 131, 129, 127, 134, 134, 134, 135, 135, 135, 135, 134, 134, 135, 134, 136, 136, 135, 135, 134, 134, 135, 134, 134, 125, 127, 122, 12

### Output your Prediction

> your filename should be **hw1_advanced.csv**

In [46]:
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in output_datalist:
    writer.writerow([row])

# Report *(5%)*

Report should be submitted as a pdf file **hw1_report.pdf**

*   Briefly describe the difficulty you encountered
*   Summarize your work and your reflections
*   No more than one page






# Save the Code File
Please save your code and submit it as an ipynb file! (**hw1.ipynb**)